# Hệ thống thông tin gợi ý bình luận 
Cấu Trúc File Code

data_loader.py: tải dữ liệu

preprocessing.py: xử lý NLP tiếng Việt

topic_modeling.py: LDA

summarizer.py: TextRank

app.py: giao diện demo hoặc lưu ra file JSON/CSV mới

# **Xây dựng các hệ thống Recommender Systems**
---

* **Hệ thống đề xuất dựa trên nội dung (Content-based recommenders):** Hệ thống này sẽ gợi ý các bình luận  

**Khảo sát dữ liệu từ file đã xử lý NLP _processed_shopee_reviews.csv**

In [100]:
# Đọc dữ liệu từ file đã upload: processed_shopee_reviews.csv
import pandas as pd

file_path = 'processed_shopee_reviews.csv'

df = pd.read_csv(file_path)
df.head(10)


,username,rating,comments,label,comments_processed,comments_clear_link,comments_clear_punctuation,comments_clear_special,comments_cleaned,comments_normalized,clean_comments
0,thanhngannehehe,5,"Áo màu sắc đẹp, vải dày dặn, đường may tinh tế...",1,"áo màu_sắc_đẹp , vải dày_dặn , đường may tinh_...","Áo màu sắc đẹp, vải dày dặn, đường may tinh tế...",Áo màu sắc đẹp vải dày dặn đường may tinh tế f...,o m u s c p v i d y d n ng may tinh t ...,o m u s c p v i d y d n ng may tinh t f...,"Áo màu sắc đẹp, vải dày dặn, đường may tinh tế...",áo vải đường may form áo kg mặc vừa nha giao h...
1,thu_nv1985,5,Chất liệu:tốt\nĐúng với mô tả:sản phẩm đúng mô...,1,chất_liệu : tốt đúng mô_tả : sản_phẩm đúng mô_...,Chất liệu:tốt\nĐúng với mô tả:sản phẩm đúng mô...,Chất liệutốt\nĐúng với mô tảsản phẩm đúng mô t...,Ch t li u t t ng v i m t s n ph m ng m ...,ch t li ut t ng v i m t s n ph m ng m t ...,Chất liệu:tốt Đúng với mô tả:sản phẩm đúng mô ...,tốt đúng đúng chuẩn đẹp hàng shop giao hàng nhanh
2,thanhthao5000,5,Màu sắc:đỏ\nĐúng với mô tả:đúng\n\nMình cái 1m...,1,màu_sắc : đỏ đúng mô_tả : đúng cái 1 m65 nặng ...,Màu sắc:đỏ\nĐúng với mô tả:đúng\n\nMình cái 1m...,Màu sắcđỏ\nĐúng với mô tảđúng\n\nMình cái 1m65...,M u s c ng v i m t ng M nh c i m ...,m u s c ng v i m t ng m nh c i m n ...,Màu sắc:đỏ Đúng với mô tả:đúng Mình cái 1m65 n...,đỏ đúng đúng mình cái nặng mặc xl vừa sít luôn...
3,thomai356,5,Màu sắc:tim than\nĐúng với mô tả:đúng với hình...,1,màu_sắc : tim than đúng mô_tả : đúng hình_ảnh ...,Màu sắc:tim than\nĐúng với mô tả:đúng với hình...,Màu sắctim than\nĐúng với mô tảđúng với hình ả...,M u s c tim than ng v i m t ng v i h nh ...,m u s ctim than ng v i m t ng v i h nh n...,Màu sắc:tim than Đúng với mô tả:đúng với hình ...,tim than đúng đúng vai gio mua chồng màu đẹp x...
4,b*****6,5,Đúng với mô tả:ok\nMàu sắc:tím than\nChất liệu...,1,đúng mô_tả : ok màu_sắc : tím than chất_liệu :...,Đúng với mô tả:ok\nMàu sắc:tím than\nChất liệu...,Đúng với mô tảok\nMàu sắctím than\nChất liệugi...,ng v i m t ok M u s c t m than Ch t li u g...,ng v i m t ok m u s ct m than ch t li ugi n...,Đúng với mô tả:ok Màu sắc:tím than Chất liệu:g...,đúng ok giống vải dù hàng yody khỏi phải nói n...
5,p*****4,5,Chất liệu:gió\nMàu sắc:ghi\nĐúng với mô tả:ok\...,1,chất_liệu : gió màu_sắc : ghi đúng mô_tả : ok ...,Chất liệu:gió\nMàu sắc:ghi\nĐúng với mô tả:ok\...,Chất liệugió\nMàu sắcghi\nĐúng với mô tảok\n\n...,Ch t li u gi M u s c ghi ng v i m t ok s...,ch t li ugi m u s cghi ng v i m t ok s n ...,Chất liệu:gió Màu sắc:ghi Đúng với mô tả:ok sả...,gió ghi đúng ok đúng vải gió giữ ấm tốt màu ck...
6,ngatran2191,5,Đúng với mô tả:đúng\nMàu sắc:xanh than\nChất l...,1,đúng mô_tả : đúng màu_sắc : xanh than chất_liệ...,Đúng với mô tả:đúng\nMàu sắc:xanh than\nChất l...,Đúng với mô tảđúng\nMàu sắcxanh than\nChất liệ...,ng v i m t ng M u s c xanh than Ch t li ...,ng v i m t ng m u s cxanh than ch t li up...,Đúng với mô tả:đúng Màu sắc:xanh than Chất liệ...,đúng đúng xanh than polyeste nhìn đường may kh...
7,u2at248y3q,5,Màu sắc:tim than\nChất liệu:mát\nĐúng với mô t...,1,màu_sắc : tim than chất_liệu : mát đúng mô_tả ...,Màu sắc:tim than\nChất liệu:mát\nĐúng với mô t...,Màu sắctim than\nChất liệumát\nĐúng với mô tảđ...,M u s c tim than Ch t li u m t ng v i m t ...,m u s ctim than ch t li um t ng v i m t n...,Màu sắc:tim than Chất liệu:mát Đúng với mô tả:...,tim than mát đúng đúng giao tới khoảng ngày mà...
8,sumisony,5,Chất liệu:ok\nMàu sắc:ok\nĐúng với mô tả:ok\n\...,1,chất_liệu : ok màu_sắc : ok đúng mô_tả : ok hà...,Chất liệu:ok\nMàu sắc:ok\nĐúng với mô tả:ok\n\...,Chất liệuok\nMàu sắcok\nĐúng với mô tảok\n\nHà...,Ch t li u ok M u s c ok ng v i m t ok H n...,ch t li uok m u s cok ng v i m t ok h ng d...,Chất liệu:ok Màu sắc:ok Đúng với mô tả:ok Hàng...,ok ok đúng ok hàng màu đẹp cản gió trượt nước ...
9,danphuong.tran,5,"Chất liệu:Đẹp , thoáng\nMàu sắc:màu đen đậm\nĐ...",1,"chất_liệu : đẹp , thoáng màu_sắc : màu đen đậm...","Chất liệu:Đẹp , thoáng\nMàu sắc:màu đen đậm\nĐ...",

Dựa vào bảng số liệu trên ta có:
*   clean_comments — đây là văn bản đã chuẩn hóa và loại bỏ ký tự thừa

Do đó, mình sẽ viết trọn bộ mã hoàn chỉnh gồm 5 mục

In [101]:
# Tạo toàn bộ mã nguồn thành các file Python lưu trong thư mục /data/project_nlp
import os

base_path = "/data/project_nlp"
os.makedirs(base_path, exist_ok=True)

files_content = {
    "data_loader.py": '''
import pandas as pd

def load_reviews(path='processed_shopee_reviews.csv'):
    df = pd.read_csv(path)
    df = df.dropna(subset=['clean_comments'])
    return df
''',

    "preprocessing.py": '''
from underthesea import word_tokenize
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\\s+', ' ', text)
    return text.strip().lower()

def tokenize_vi(text):
    return word_tokenize(text, format="text").split()

def preprocess_reviews(df, column='clean_comments'):
    df['cleaned'] = df[column].apply(clean_text)
    df['tokens'] = df['cleaned'].apply(tokenize_vi)
    return df
''',

    "topic_modeling.py": '''
from gensim import corpora, models
import pyLDAvis.gensim_models
import pyLDAvis

def build_lda_model(token_lists, num_topics=5, passes=10):
    dictionary = corpora.Dictionary(token_lists)
    corpus = [dictionary.doc2bow(text) for text in token_lists]
    lda_model = models.LdaModel(corpus=corpus, id2word=dictionary,
                                num_topics=num_topics, passes=passes,
                                random_state=42)
    return lda_model, corpus, dictionary

def show_topics(lda_model, num_words=10):
    topics = lda_model.print_topics(num_words=num_words)
    for topic in topics:
        print(topic)

def visualize_lda(lda_model, corpus, dictionary):
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(vis, 'lda_vietnamese.html')
''',

    "summarizer.py": '''
from summa.summarizer import summarize

def summarize_review(text, ratio=0.3):
    try:
        return summarize(text, ratio=ratio)
    except:
        return ""

def summarize_all(df, column='clean_comments'):
    df['summary'] = df[column].apply(lambda x: summarize_review(x, ratio=0.3))
    return df
''',

    "main.py": '''
from data_loader import load_reviews
from preprocessing import preprocess_reviews
from topic_modeling import build_lda_model, show_topics, visualize_lda
from summarizer import summarize_all

def main():
    df = load_reviews('processed_shopee_reviews.csv')
    df = preprocess_reviews(df, column='clean_comments')

    lda_model, corpus, dictionary = build_lda_model(df['tokens'].tolist(), num_topics=5)
    show_topics(lda_model)
    visualize_lda(lda_model, corpus, dictionary)

    df = summarize_all(df, column='clean_comments')
    df[['clean_comments', 'summary']].to_csv('output_summary.csv', index=False)
    print(" Hoàn thành. File kết quả: output_summary.csv, LDA visualization: lda_vietnamese.html")

if __name__ == '__main__':
    main()
''',

    "requirements.txt": '''
pandas
underthesea
gensim
summa
pyLDAvis
'''
}

# Ghi từng file vào thư mục
file_paths = []
for filename, content in files_content.items():
    full_path = os.path.join(base_path, filename)
    with open(full_path, "w", encoding="utf-8") as f:
        f.write(content.strip())
    file_paths.append(full_path)

file_paths


<>:17: DeprecationWarning: invalid escape sequence '\w'
<>:17: DeprecationWarning: invalid escape sequence '\w'
C:\Users\DINH THI TRANG\AppData\Local\Temp\ipykernel_28332\2152127937.py:17: DeprecationWarning: invalid escape sequence '\w'
  "preprocessing.py": '''


['/data/project_nlp\\data_loader.py',
 '/data/project_nlp\\preprocessing.py',
 '/data/project_nlp\\topic_modeling.py',
 '/data/project_nlp\\summarizer.py',
 '/data/project_nlp\\main.py',
 '/data/project_nlp\\requirements.txt']

* Di chuyển file processed_shopee_reviews.csv vào cùng thư mục chứa mã nguồn.

* Cài thư viện:
    pip install -r requirements.txt

* Chạy main.py:
    python main.py

In [102]:
import shutil

# Nén toàn bộ thư mục chứa mã nguồn thành file zip
zip_path = "/data/project_nlp.zip"
shutil.make_archive(base_name=zip_path.replace(".zip", ""), format='zip', root_dir="/data/project_nlp")

zip_path
# Tải file zip về máy
import shutil  

# **Phân tích chủ đề (Topic Modeling với LDA)**
* 1.Dùng thư viện gensim để huấn luyện mô hình LDA:

**Xử lý văn bản từ file processed_shopee_reviews có thể dừng thư viện underthesea hoặc VnCorreNPL để xử lý tiếng việt**

* data_loader
* Preprocessing
* Summarizer
    * Xử lý văn bản ngắn
    * Xử lý ngoại lệ
    
        Kiểm tra độ dài văn bản: Nếu văn bản có ít hơn 5 từ, không áp dụng summarize, tránh trường hợp không thể tóm tắt văn bản quá ngắn.

        Xử lý lỗi cụ thể hơn: In ra thông báo lỗi chi tiết nếu có sự cố khi gọi summarize
* topic_moderling
* main

 **Tóm tắt tự động với TextRank**

In [103]:
# 1. IMPORT THƯ VIỆN
import pandas as pd
import os
import re
from underthesea import word_tokenize
from summa.summarizer import summarize
from gensim import corpora, models
import pyLDAvis.gensim_models
import pyLDAvis



In [104]:


# 2. LOAD DỮ LIỆU
def load_reviews(path='processed_shopee_reviews.csv'):
    if not os.path.exists(path):
        print(f" File {path} không tồn tại.")
        return None
    
    df = pd.read_csv(path)
    if 'clean_comments' not in df.columns:
        print(" Cột 'clean_comments' không tồn tại trong dữ liệu.")
        return None
    
    df = df.dropna(subset=['clean_comments'])
    print(f" Đã tải {len(df)} dòng dữ liệu từ {path}")
    return df

In [105]:
# 3. TIỀN XỬ LÝ VĂN BẢN của bình luận
def clean_text(text):
    text = re.sub(r'[^\w\s]', ' ', text)     # Xoá ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text)         # Xoá khoảng trắng thừa
    return text.strip().lower()

def tokenize_vi(text):
    return word_tokenize(text, format="text").split()

def preprocess_reviews(df, column='clean_comments'):
    df['cleaned'] = df[column].apply(clean_text)
    df['tokens'] = df['cleaned'].apply(tokenize_vi)
    print(" Đã tiền xử lý xong văn bản.")
    return df

# 4. MÔ HÌNH CHỦ ĐỀ LDA
def build_lda_model(token_lists, num_topics=5, passes=10):
    dictionary = corpora.Dictionary(token_lists)
    corpus = [dictionary.doc2bow(text) for text in token_lists]
    lda_model = models.LdaModel(corpus=corpus,
                                id2word=dictionary,
                                num_topics=num_topics,
                                passes=passes,
                                random_state=42)
    return lda_model, corpus, dictionary

def show_topics(lda_model, num_words=10):
    topics = lda_model.print_topics(num_words=num_words)
    print("\n Các chủ đề được trích xuất:")
    for idx, topic in topics:
        print(f"Chủ đề {idx + 1}: {topic}")

def visualize_lda(lda_model, corpus, dictionary, filename='lda_vietnamese.html'):
    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
    pyLDAvis.save_html(vis, filename)
    print(f" Đã lưu kết quả trực quan hóa vào: {filename}")



In [ ]:

# 5. TÓM TẮT BÌNH LUẬN
def summarize_review(text, ratio=0.3):
    if len(text.split()) < 5:
        return text
    try:
        return summarize(text, ratio=ratio)
    except Exception as e:
        print(f" Lỗi khi tóm tắt: {e}")
        return ""

def summarize_all(df, column='clean_comments'):
    df['summary'] = df[column].apply(lambda x: summarize_review(x, ratio=0.3))
    print(" Đã tóm tắt tất cả bình luận.")
    return df


In [ ]:
# 6. CHẠY TOÀN BỘ QUY TRÌNH
def main():
    df = load_reviews('processed_shopee_reviews.csv')
    if df is None or df.empty:
        print(" Không thể xử lý vì dữ liệu bị thiếu hoặc rỗng.")
        return

    df = preprocess_reviews(df, column='clean_comments')

    lda_model, corpus, dictionary = build_lda_model(df['tokens'].tolist(), num_topics=5)
    show_topics(lda_model)
    visualize_lda(lda_model, corpus, dictionary)

    df = summarize_all(df, column='clean_comments')

    df[['clean_comments', 'summary']].to_csv('output_summary.csv', index=False)
    print(" Hoàn thành toàn bộ pipeline.")
    print(" File kết quả: output_summary.csv")
    print(" File visualization: lda_vietnamese.html")

# 7. CHẠY CHƯƠNG TRÌNH

main()


 Đã tải 1175 dòng dữ liệu từ processed_shopee_reviews.csv
 Đã tiền xử lý xong văn bản.

 Các chủ đề được trích xuất:
Chủ đề 1: 0.042*"nhận" + 0.038*"mang" + 0.033*"áo" + 0.025*"xu" + 0.023*"đẹp" + 0.018*"chỉ" + 0.017*"đúng" + 0.016*"mua" + 0.011*"hàng" + 0.010*"yody"
Chủ đề 2: 0.012*"ngay" + 0.010*"ok" + 0.009*"đường" + 0.009*"goi" + 0.008*"mang" + 0.007*"đúng" + 0.007*"chi" + 0.007*"de" + 0.007*"viettel" + 0.006*"tran"
Chủ đề 3: 0.088*"hàng" + 0.056*"giao" + 0.049*"nhanh" + 0.042*"đúng" + 0.031*"shop" + 0.030*"đẹp" + 0.021*"lần" + 0.018*"sẽ" + 0.017*"mua" + 0.013*"tốt"
Chủ đề 4: 0.084*"đúng" + 0.038*"mua" + 0.032*"ok" + 0.027*"đẹp" + 0.025*"áo" + 0.025*"hàng" + 0.019*"tốt" + 0.017*"shop" + 0.016*"yody" + 0.015*"giao"
Chủ đề 5: 0.048*"đẹp" + 0.043*"áo" + 0.037*"mua" + 0.026*"đúng" + 0.025*"mặc" + 0.021*"mình" + 0.020*"giá" + 0.016*"size" + 0.012*"hơn" + 0.012*"màu"
 Đã lưu kết quả trực quan hóa vào: lda_vietnamese.html
 Đã tóm tắt tất cả bình luận.
 Hoàn thành toàn bộ pipeline.
 File k

In [107]:
def get_dominant_topic(doc_bow):
    topic_probs = lda_model.get_document_topics(doc_bow)
    if topic_probs:
        dominant = sorted(topic_probs, key=lambda x: -x[1])[0]
        return dominant[0]
    return None

def suggest(topic_id):
    if topic_id is None:
        return "Không xác định được chủ đề"
    keywords = lda_model.print_topic(topic_id, topn=3)
    return f"Review này đề cập đến {keywords}"

df['dominant_topic'] = [get_dominant_topic(bow) for bow in corpus]
df['suggestion'] = df['dominant_topic'].apply(suggest)
df[['clean_comments', 'dominant_topic', 'suggestion']].to_csv('output_suggestion.csv', index=False)
print(" Hoàn thành. File kết quả: output_suggestion.csv")

NameError: name 'corpus' is not defined

In [108]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

# Word Cloud
all_words = ' '.join([' '.join(tokens) for tokens in df['tokens']])
wordcloud = WordCloud(width=800, height=400, background_color='white', font_path='arial.ttf').generate(all_words)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Word Cloud các từ phổ biến")
plt.show()

# Biểu đồ số lượng chủ đề
plt.figure(figsize=(8, 4))
sns.countplot(x='dominant_topic', data=df)
plt.title("Phân phối các chủ đề trong review")
plt.xlabel("Chủ đề")
plt.ylabel("Số lượng")
plt.show()


KeyError: 'tokens'

In [110]:
df.to_csv("final_reviews_with_topics.csv", index=False)

# **Trực quan hóa & Gợi ý**

Hiển thị các chủ đề phổ biến dưới dạng word cloud hoặc biểu đồ.

Với mỗi review, hiển thị:

*   Chủ đề chính.
*   Tóm tắt nội dung.
*   Gợi ý cho người dùng (ví dụ: "Review này đề cập đến chất lượng sản phẩm và giao hàng").

In [ ]:
# Cài đặt thư viện nếu cần
# !pip install pandas matplotlib seaborn wordcloud gensim nltk scikit-learn

import os
import pandas as pd
import string
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\DINH THI
[nltk_data]     TRANG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\DINH THI
[nltk_data]     TRANG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import os

# Tạo nội dung stopwords tiếng Việt
stopwords_vi = """
và
là
của
có
cho
một
những
các
đã
đến
trong
khi
với
này
rằng
vì
nên
thì
lại
đó
từ
ra
được
tại
theo
nó
như
đây
nào
cũng
rất
còn
bị
trên
dưới
giữa
vẫn
sẽ
đi
lúc
hơn
nữa
hay
cả
ai
gì
đâu
vậy
ấy
thôi
luôn
vừa
mới
đang
vào
kể
nếu
hoặc
để
mà
nhưng
rồi
do
thế
chỉ
kia
""".strip()

# Đảm bảo thư mục tồn tại
folder_path = "data"
os.makedirs(folder_path, exist_ok=True)

# Lưu file vào thư mục
file_path = os.path.join(folder_path, "vietnamese_stopwords.txt")
with open(file_path, "w", encoding="utf-8") as f:
    f.write(stopwords_vi)

print("File đã được lưu tại:", file_path)


File đã được lưu tại: data\vietnamese_stopwords.txt


In [ ]:
# Đọc danh sách stopwords từ file
with open("vietnamese_stopwords.txt", encoding="utf-8") as f:
    stop_words = set([line.strip() for line in f.readlines()])


In [ ]:

from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download("punkt")

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [t for t in tokens if t not in stop_words and t not in string.punctuation and t.isalpha()]
    return tokens

df['tokens'] = df['clean_comments'].apply(preprocess)


[nltk_data] Downloading package punkt to C:\Users\DINH THI
[nltk_data]     TRANG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
from gensim import corpora, models

# Tạo từ điển và tập văn bản
dictionary = corpora.Dictionary(df['tokens'])
corpus = [dictionary.doc2bow(text) for text in df['tokens']]

# Train mô hình LDA
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)

# In ra các chủ đề
for i, topic in lda_model.print_topics(num_words=5):
    print(f"🔸 Chủ đề {i}: {topic}")


KeyError: 'tokens'

In [ ]:
def generate_suggestions(df):
    # Gợi ý dựa trên chủ đề hoặc các từ khóa chính
    df['suggestions'] = df['summary'].apply(lambda x: "Review này đề cập đến chất lượng sản phẩm và giao hàng" if 'chất lượng' in x else "Review này đề cập đến các vấn đề khác.")
    return df
